# RGI scratchpad

In [3]:
import random
from rgi.games.count21.count21 import Count21Game

In [ ]:
game = Count21Game()
state = game.initial_state()

print(f'state:   {game.pretty_str(state)}')

while not game.is_terminal(state):
    legal_actions = game.legal_actions(state)
    action = random.choice(legal_actions)
    print(f'action: {action}')
    print(f'\nstate:   {game.pretty_str(state)}')
    state = game.next_state(state, action)

print(f'\nfinal state:   {game.pretty_str(state)}')
print(f'reward[1]:     {game.reward(state, 1)}')
print(f'reward[2]:     {game.reward(state, 2)}')

In [ ]:
import importlib
import numpy as np

from rgi.games.connect4 import connect4
importlib.reload(connect4)

game = connect4.Connect4Game()
s1 = game.initial_state()

s = s1
for action in [1,2,1,2,1,2,1,6,6,6,6,6]:
    s = game.next_state(s, action)
    print(f'\naction={action}, next state:\n{game.pretty_str(s)}\nresult={game.reward(s, 1)},{game.reward(s, 2)}')

np.all(s.board != 0).item()